## Punto dos Laboratorio 5 MOS
- Samuel Ramirez Gómez
- Paulina Arrazzola

# Conjuntos 
- Recursos
- Zona Destino
- Viajes
- Avion


# Indices

- $r \in R$ Recursos
- $a \in A$ Aviones
- $v \in V$ Viajes
- $d \in ZD$ Zona destino

In [ ]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import os
import numpy as np
import pandas as pd
from pyomo.environ import value
numav = 4
numrec = 5
numzonas = 4
numviajes = 2
model = ConcreteModel()
model.R = RangeSet(numrec)
model.V = RangeSet(numviajes)
model.Z = RangeSet(numzonas)
model.A = RangeSet(numav)

# Parametros

Recursos:
- $I_r$ El impacto que tiene cada recurso r al ser entregado
- $P_r$ El peso en toneladas por unidad del recurso r
- $Vo_r$ El volumen en $m^3$ por unidad del recurso r
- $Di_r$ Disponibilidad en unidades del recurso r

Aviones:
- $C_a$ La capacidad de peso en toneladas del avion a
- $C_v$ La capacidad de volumen en $m^3$ del avion a
- $F_a$ El costo fijo que tiene usar el avion a
- $K_a$ El costo variable que tiene el avion a

Zonas:
- $D_d$ La distancia que hay hasta la zona destino d
- $Mi_d$ El multiplicador de impacto que tiene cada zona destino d

Matriz Necesidades:
- $N_{dr}$ La necesidad en unidades de recurso r para la zona destino d 

In [2]:
# Recursos
Impacto = {1:50,2:100,3:60,4:40,5:120}
Peso = {1:5,2:2,3:6,4:3,5:0.3}
Volumen = {1:3,2:1,3:4,4:2,5:0.5}
Disponibilidad = {1:12,2:15,3:15,4:20,5:40}
model.I = Param(model.R,initialize=Impacto)
model.P = Param(model.R,initialize=Peso)
model.Vo = Param(model.R,initialize=Volumen)
model.Di = Param(model.R,initialize=Disponibilidad)


#Aviones 
capacidadp = {1:40,2:50,3:60,4:45}
capacidadv = {1:35,2:40,3:45,4:38}
costofijo = {1:15,2:20,3:25,4:18}
costovar = {1:0.020,2:0.025,3:0.03,4:0.022}
model.C = Param(model.A,initialize=capacidadp)
model.Cv = Param(model.A,initialize=capacidadv)
model.F = Param(model.A,initialize=costofijo)
model.K = Param(model.A,initialize=costovar)

#Zonas
distancia = {1:800,2:1200,3:1500,4:900}
multiplicador = {1:1.2,2:1.5,3:1.8,4:1.4}
model.D = Param(model.Z,initialize=distancia)
model.Mi = Param(model.Z,initialize=multiplicador)

#Matriz necesidades
necesidades = {(1,1):8,(1,2):2,(1,3):6,(1,4):3,(1,5):0.6,
               (2,1):12,(2,2):3,(2,3):9,(2,4):5,(2,5):0.9,
                (3,1):16,(3,2):4,(3,3):12,(3,4):7,(3,5):1.2,
                 (4,1):10,(4,2):2,(4,3):8,(4,4):4,(4,5):0.6 }

model.N = Param(model.Z,model.R,initialize=necesidades)




# Variables de decisión

- $x_{radv}$ cantidad de unidades del recurso r que transporta el avion a hacia el destino d en el viaje v (real no negativa)

- $xm_{adv}$ variable auxiliar entera para verificar el transporte de medicamentos en el avion a hacia el destino d en el viaje v (entera no negativa)

- $y_{avd}$ variable auxiliar para determinar que el avion a viaja al destino d en el viaje v (binaria)

- $u_{a}$ variable auxiliar para determinar si el avion a fue utilizado (binaria)

- $z_{adv}$ variable auxiliar para modelar la exlcusividad de transporte de equipos medicos y agua 

In [3]:

model.x = Var(model.R,model.A,model.Z,model.V, within = NonNegativeReals)
model.xm = Var(model.A,model.Z,model.V,within= NonNegativeIntegers)
model.y= Var(model.A,model.V,model.Z, within= Binary)
model.u = Var(model.A,within=Binary)
model.me = Var(model.A,model.Z,model.V,within=PositiveIntegers)
model.z = Var(model.A,model.Z,model.V, within=Binary)


# Funciones objetivo

- Se plantean ambas funciones objetivo una para costos y otra para el impacto
- Funcion impacto = $$\sum_{r \in R} \sum_{a \in A} \sum_{d \in ZD} \sum_{v \in V} x_{radv}*I_{r}*Mi_{d}  $$
- Funcion costo = $$\sum_{a \in A} u_{a}*F_{a} + \sum_{a \in A} \sum_{d \in ZD} \sum_{v \in V} y_{adv}*D_{d}*K_{a}  $$

In [4]:
#Impacto
model.obj = Objective(expr= sum(model.x[r,a,d,v]*model.I[r]*model.Mi[d] for r in model.R for a in model.A for d in model.Z for v in model.V) ,sense=maximize)

model.del_component('obj')

#Costo
model.obj = Objective(expr = sum(model.u[a]*model.F[a] for a in model.A) + sum(model.y[a,v,d]*model.D[d]*model.K[a] for a in model.A for v in model.V for d in model.Z),sense=minimize)

# Restricciones

Restriccion de capacidad y volumen:
- $$ \sum_{r \in R} x_{radv}*P_r \leq C_a \quad  \forall a \in A \quad \forall d \in ZD \quad \forall v \in V$$
- $$ \sum_{r \in R} x_{radv}*Vo_r \leq Cv_a \quad  \forall a \in A \quad \forall d \in ZD \quad \forall v \in V$$

Restriccion de necesidad:
- $$ \sum_{a \in A} \sum_{v \in V} x_{radv}*P_r  \geq N_{dr} \quad \forall d \in ZD \quad \forall r \in R $$

Restriccion de disponibilidad:
- $$ \sum_{a \in A} \sum_{v \in V} \sum_{d \in ZD} x_{radv}  \leq Di_{r} \quad  \forall r \in R $$

Restriccion logica medicinas viajando en el avion 1
- $$ x_{med, 1, d, v} = 0 \quad \forall d \in ZD \quad \forall v \in V$$

Restriccion un avion solo va a un destino por viaje
- $$ \sum_{d \in ZD} y_{avd} \leq 1 \quad  \forall a \in A \quad \forall v \in V$$

Restriccion de agua y equipos medicos no viajan en un mismo avion
- $$ x_{eqmed,a,d,v} \leq 100*z_{a,d,v} \quad \forall a \in A \quad \forall v \in V \quad \forall d \in ZD$$
- $$ x_{ag,a,d,v} \leq 100(1-*z_{a,d,v}) \quad \forall a \in A \quad \forall v \in V \quad \forall d \in ZD$$

Restriccion de llevar unidades enteras de equipos medicos
- $$ x_{eqmed,a,d,v} = xm_{adv} \quad \forall a \in A \quad \forall v \in V \quad \forall d \in ZD$$

Restricciones lógicas para activacion de variables
- $$ y_{avd} \leq u_a \quad \forall a \in A \quad \forall v \in V \quad \forall d \in ZD$$
- $$ x_{radv} \leq 1000*y_{avd} \quad \forall a \in A \quad \forall v \in V \quad \forall d \in ZD \quad \forall r \in R$$

In [5]:
model.restricciones = ConstraintList()

for a in model.A:
    for d in model.Z:
        for v in model.V:
           model.restricciones.add(sum(model.x[r,a,d,v]*model.P[r] for r in model.R)  <= model.C[a])
           
           model.restricciones.add(sum(model.x[r,a,d,v]*model.Vo[r] for r in model.R) <= model.Cv[a])
          


for d in model.Z:
    for r in model.R:
       model.restricciones.add(sum(model.x[r,a,d,v]*model.P[r] for a in model.A for v in model.V) >= model.N[d,r])

for r in model.R:
    model.restricciones.add(sum(model.x[r,a,d,v] for a in model.A for d in model.Z for v in model.V) <= model.Di[r])


for d in model.Z:
    for v in model.V:
        model.restricciones.add(model.x[2,1,d,v] == 0)

for a in model.A:
    for v in model.V:
       model.restricciones.add(sum(model.y[a,v,d] for d in model.Z) <= 1)
       for d in model.Z:
            model.restricciones.add(model.x[5,a,d,v]  <= 100*model.z[a,d,v])
            model.restricciones.add(model.x[3,a,d,v] <= 100*(1 - model.z[a,d,v]) )
            model.restricciones.add((model.x[5,a,d,v]) == model.xm[a,d,v])
            
            


for a in model.A:
    for v in model.V:
        for d in model.Z:
           model.restricciones.add(model.y[a,v,d] <= model.u[a] )
          
           for r in model.R:
             model.restricciones.add(model.x[r,a,d,v] <= model.y[a,v,d]*1000)









# Maximo Impacto

In [6]:
model.del_component('obj')
model.obj = Objective(expr= sum(model.x[r,a,d,v]*model.I[r]*model.Mi[d] for r in model.R for a in model.A for d in model.Z for v in model.V) ,sense=maximize)

solver = SolverFactory('glpk')

result = solver.solve(model)
impactomax = float(model.obj())

print(f"maximo impacto = {model.obj()}")


maximo impacto = 14526.066666666662


## Minimo Impacto

In [7]:
model.del_component('obj')
model.obj = Objective(expr= sum(model.x[r,a,d,v]*model.I[r]*model.Mi[d] for r in model.R for a in model.A for d in model.Z for v in model.V) ,sense=minimize)

solver = SolverFactory('glpk')

result = solver.solve(model)
impactomin = float(model.obj())

print(f"Minimo impacto = {model.obj()}")

Minimo impacto = 4502.666666666664


## Maximo costo

In [8]:
model.del_component('obj')
model.obj = Objective(expr = sum(model.u[a]*model.F[a] for a in model.A) + sum(model.y[a,v,d]*model.D[d]*model.K[a] for a in model.A for v in model.V for d in model.Z),sense=maximize)

solver = SolverFactory('glpk')

result = solver.solve(model)
costomax = float(model.obj())

print(f"Maximo costo = {model.obj()}")


Maximo costo = 299.40000000000003


## Minimo costo

In [9]:
model.del_component('obj')
model.obj = Objective(expr = sum(model.u[a]*model.F[a] for a in model.A) + sum(model.y[a,v,d]*model.D[d]*model.K[a] for a in model.A for v in model.V for d in model.Z),sense=minimize)

solver = SolverFactory('glpk')

result = solver.solve(model)
costomin = float(model.obj())

print(f"Minimo costo = {model.obj()}")

Minimo costo = 284.4


## Función objetivo redefinida y frente de pareto

$$ w_1*\frac{\sum_{r \in R} \sum_{a \in A} \sum_{d \in ZD} \sum_{v \in V} x_{radv}*I_{r}*Mi_{d} - 4502.6}{14526 - 4502.6}$$ 
$$ + w_2*\frac{\sum_{a \in A} u_{a}*F_{a} + \sum_{a \in A} \sum_{d \in ZD} \sum_{v \in V} y_{adv}*D_{d}*K_{a} - 284.4}{299.4 - 284.4}$$

In [25]:
frente_pareto = np.zeros((1,10),dtype=float)
impactovscosto =  np.zeros((1,10),dtype=float) 
for i in range(1,10):
     
      w1 = i*0.1
      w2 = (10-i)*0.1
      model.del_component('obj')
      model.obj = Objective(expr = w1*(sum(model.x[r,a,d,v]*model.I[r]*model.Mi[d] for r in model.R for a in model.A for d in model.Z for v in model.V)-4502.6)/(14526.066-4502.6) + w2*(1- ((sum(model.u[a]*model.F[a] for a in model.A) + sum(model.y[a,v,d]*model.D[d]*model.K[a] for a in model.A for v in model.V for d in model.Z) - 284.4)/(299.4 - 284.4))) , sense= maximize)
      solver = SolverFactory('gurobi')
 
      result = solver.solve(model)
      frente_pareto[0,i] = float(model.obj())
      valor_costonorm = w2*float(model.obj())
      valor_impactonorm = w1*float(model.obj())
      valor_costo = valor_costonorm*(costomax-costomin)  + costomin
      valor_impacto = valor_impactonorm*(impactomax-impactomin) + impactomin
      if valor_costo != 0:
        relacionimpactocosto = valor_impacto/valor_costo
        impactovscosto[0,i] = relacionimpactocosto


print (f"Frente de Pareto = {frente_pareto} ")
print(f"Relacion impacto/costo = {impactovscosto}")      

Frente de Pareto = [[0.         0.99745298 0.99490597 0.99235895 0.98981193 0.98726492
  0.9847179  0.98217088 0.97962387 0.97707685]] 
Relacion impacto/costo = [[ 0.         18.47294017 21.92467942 25.3941922  28.88148351 32.38655565
  35.90940819 39.45003785 43.00843843 46.58460079]]


Se puede observar que bajo el criterio de relacion impacto vs costo la mejor configuración para los pesos es la inversa a la configuración inversa a la configuración del frente de pareto. Con w1 siendo igual a 0.9 y w2 siendo igual a 0.1 

In [26]:
w1 = 9*0.1
w2 = (10-9)*0.1
model.del_component('obj')
model.obj = Objective(expr = w1*(sum(model.x[r,a,d,v]*model.I[r]*model.Mi[d] for r in model.R for a in model.A for d in model.Z for v in model.V)-4502.6)/(14526.066-4502.6) + w2*(1- ((sum(model.u[a]*model.F[a] for a in model.A) + sum(model.y[a,v,d]*model.D[d]*model.K[a] for a in model.A for v in model.V for d in model.Z) - 284.4)/(299.4 - 284.4))) , sense= maximize)
solver = SolverFactory('gurobi')
result = solver.solve(model)

data = []

for r in model.R:
    for a in model.A:
        for d in model.D:
            for v in model.V:
                val = value(model.x[r, a, d, v].value)
                if val > 0:
                    data.append({
                        'Recurso': r,
                        'Avión': a,
                        'Destino': d,
                        'Viaje': v,
                        'Unidades': val
                    })

df = pd.DataFrame(data)
print(df)
df.sort_values(by="Destino")


    Recurso  Avión  Destino  Viaje   Unidades
0         1      1        3      2   3.200000
1         1      2        4      2   2.000000
2         1      3        1      2   2.400000
3         1      4        2      2   4.400000
4         2      2        4      2   1.000000
5         2      3        1      2  10.500000
6         2      4        2      2   1.500000
7         2      4        3      1   2.000000
8         3      1        2      1   6.050000
9         3      1        3      2   2.000000
10        3      2        4      1   1.333333
11        3      3        1      1   5.616667
12        4      2        4      1   1.333333
13        4      3        1      1   8.766667
14        4      3        1      2   5.900000
15        4      4        2      2   1.666667
16        4      4        3      1   2.333333
17        5      2        4      2   2.000000
18        5      3        1      2  31.000000
19        5      4        2      2   3.000000
20        5      4        3      1

,Recurso,Avión,Destino,Viaje,Unidades
2,1,3,1,2,2.400000
5,2,3,1,2,10.500000
13,4,3,1,1,8.766667
11,3,3,1,1,5.616667
14,4,3,1,2,5.900000
18,5,3,1,2,31.000000
6,2,4,2,2,1.500000
8,3,1,2,1,6.050000
15,4,4,2,2,1.666667
3,1,4,2,2,4.400000


Se observa que la distribución de los recursos en los destinos cambia drasticamente. Solo analizando el destino uno que en la solución optima tiene una distribución de:
Para el destino 1 
- Recurso 1: 1.6
- Recurso 2: 1
- Recurso 3: 1
- Recurso 4: 1
- Recurso 5: 2

En cambio para la distribución con mejor relación impacto/costo, la distribución siguente.
Para el destino 1 
- Recurso 1: 2.4
- Recurso 2: 10.5
- Recurso 3: 5.61
- Recurso 4: 14.66
- Recurso 5: 31


# Valor optimo Frente de pareto

In [11]:
Valor_max = np.max(frente_pareto)
indicemax = np.argmax(frente_pareto)

print(f"Valor objetivo maximo = {Valor_max}")
print(f"Valor de w1 = {indicemax*0.1}")
print(f"Valor de w2 = {(10-indicemax)*0.1}")

Valor objetivo maximo = 0.9983658413832767
Valor de w1 = 0.1
Valor de w2 = 0.9


# Visualización de resultados


In [12]:
w1 = 0.1
w2 = 1-w1
model.del_component('obj')
model.obj = Objective(expr = w1*(sum(model.x[r,a,d,v]*model.I[r]*model.Mi[d] for r in model.R for a in model.A for d in model.Z for v in model.V)-4502.6)/(14526.066-4502.6) + w2*(1- ((sum(model.u[a]*model.F[a] for a in model.A) + sum(model.y[a,v,d]*model.D[d]*model.K[a] for a in model.A for v in model.V for d in model.Z) - 284.4)/(299.4 - 284.4))) , sense= maximize)
result = solver.solve(model)

In [ ]:


data = []

for r in model.R:
    for a in model.A:
        for d in model.D:
            for v in model.V:
                val = value(model.x[r, a, d, v].value)
                if val > 0:
                    data.append({
                        'Recurso': r,
                        'Avión': a,
                        'Destino': d,
                        'Viaje': v,
                        'Unidades': val
                    })

df = pd.DataFrame(data)
print(df)
df.sort_values(by="Destino")

    Recurso  Avión  Destino  Viaje      Unidades
0         1      1        3      1  3.200000e+00
1         1      2        4      1  1.160000e+00
2         1      2        4      2  3.600000e+00
3         1      3        1      2  1.600000e+00
4         1      4        2      1  2.440000e+00
5         2      2        4      1  1.000000e+00
6         2      3        1      1  1.000000e+00
7         2      4        2      1  1.500000e+00
8         2      4        3      2  1.150000e+01
9         3      1        2      2  6.666667e+00
10        3      1        3      1  2.000000e+00
11        3      2        4      2  5.333333e+00
12        3      3        1      2  1.000000e+00
13        4      1        3      1  4.000000e+00
14        4      2        4      1  1.333333e+00
15        4      3        1      1  1.000000e+00
16        4      4        2      1  9.633333e+00
17        4      4        3      2  4.033333e+00
18        5      1        2      2  3.700743e-16
19        5      2  

,Recurso,Avión,Destino,Viaje,Unidades
3,1,3,1,2,1.600000e+00
6,2,3,1,1,1.000000e+00
12,3,3,1,2,1.000000e+00
15,4,3,1,1,1.000000e+00
20,5,3,1,1,2.000000e+00
4,1,4,2,1,2.440000e+00
7,2,4,2,1,1.500000e+00
9,3,1,2,2,6.666667e+00
18,5,1,2,2,3.700743e-16
21,5,4,2,1,3.000000e+00


# Ordenados por destino

In [14]:
print(df)

    Recurso  Avión  Destino  Viaje      Unidades
0         1      1        3      1  3.200000e+00
1         1      2        4      1  1.160000e+00
2         1      2        4      2  3.600000e+00
3         1      3        1      2  1.600000e+00
4         1      4        2      1  2.440000e+00
5         2      2        4      1  1.000000e+00
6         2      3        1      1  1.000000e+00
7         2      4        2      1  1.500000e+00
8         2      4        3      2  1.150000e+01
9         3      1        2      2  6.666667e+00
10        3      1        3      1  2.000000e+00
11        3      2        4      2  5.333333e+00
12        3      3        1      2  1.000000e+00
13        4      1        3      1  4.000000e+00
14        4      2        4      1  1.333333e+00
15        4      3        1      1  1.000000e+00
16        4      4        2      1  9.633333e+00
17        4      4        3      2  4.033333e+00
18        5      1        2      2  3.700743e-16
19        5      2  

# Analisis 
- Siguiendo uno de los puntos se va a plantear que una solución sea 5 veces mas importante que la otra. Para este caso se definirá w1 = 0.165 y w2 = 0.825

In [15]:
w1 = 0.1666666666666666666
w2 = 1-w1
model.del_component('obj')
model.obj = Objective(expr = w1*(sum(model.x[r,a,d,v]*model.I[r]*model.Mi[d] for r in model.R for a in model.A for d in model.Z for v in model.V)-4502.6)/(14526.066-4502.6) + w2*(1- ((sum(model.u[a]*model.F[a] for a in model.A) + sum(model.y[a,v,d]*model.D[d]*model.K[a] for a in model.A for v in model.V for d in model.Z) - 284.4)/(299.4 - 284.4))) , sense= maximize)
result = solver.solve(model)

In [16]:

data = []

for r in model.R:
    for a in model.A:
        for d in model.D:
            for v in model.V:
                val = value(model.x[r, a, d, v].value)
                if val > 0:
                    data.append({
                        'Recurso': r,
                        'Avión': a,
                        'Destino': d,
                        'Viaje': v,
                        'Unidades': val
                    })

df = pd.DataFrame(data)
print(df)

    Recurso  Avión  Destino  Viaje   Unidades
0         1      1        2      2   2.760000
1         1      1        3      1   0.780000
2         1      2        4      2   2.000000
3         1      3        1      2   1.600000
4         1      4        2      1   2.440000
5         1      4        3      2   2.420000
6         2      2        4      2   1.000000
7         2      3        1      2   1.000000
8         2      4        2      1   1.500000
9         2      4        3      2  11.500000
10        3      1        2      2   4.366667
11        3      1        3      1   2.000000
12        3      2        4      1   7.633333
13        3      3        1      2   1.000000
14        4      1        3      1   8.033333
15        4      2        4      2   1.333333
16        4      3        1      1   1.000000
17        4      4        2      1   9.633333
18        5      2        4      2   2.000000
19        5      3        1      1   2.000000
20        5      4        2      1

Observando ambas tablas cambiando los pesos de las soluciones para que una función sea 5 veces más importante que la otra se pueden observar grandes cambios en las soluciones. Por ejemplo se observa como para el recurso 1 avión 1 destino 3 en el viaje uno para la solución con pesos w1 = 0.1 y w2 = 0.9 esta tiene un valor de 3.2 unidades. Mientras que para esta misma variable aleatoria con los pesos w1 = 0.1666666 y w2 = 0.833333, este valor cae hasta 0.78 unidades. Se puede observar como cambios en los pesos de las funciones objetivo resultan en variaciones considerables en la distribución y asignacion de recursos, vuelos, viajes y destinos. Es importante elegir los pesos de un problema de suma ponderada dependiendo del objetivo de la optimización. Se pueden escoger los pesos para obtener el mejor valor objetivo modelado. O como en este laboratorio para obtener la mejor relación impacto beneficio.

# Cambio de multiplicadores 


In [17]:
model.del_component('Mi')
multiplicador = {1:1.8,2:1.5,3:1.2,4:1.4}
model.Mi = Param(model.Z,initialize=multiplicador)
result = solver.solve(model)

data = []

for r in model.R:
    for a in model.A:
        for d in model.D:
            for v in model.V:
                val = value(model.x[r, a, d, v].value)
                if val > 0:
                    data.append({
                        'Recurso': r,
                        'Avión': a,
                        'Destino': d,
                        'Viaje': v,
                        'Unidades': val
                    })

df = pd.DataFrame(data)


print(df)
df.sort_values(by="Destino")


    Recurso  Avión  Destino  Viaje   Unidades
0         1      1        2      2   2.760000
1         1      1        3      1   0.780000
2         1      2        4      2   2.000000
3         1      3        1      2   1.600000
4         1      4        2      1   2.440000
5         1      4        3      2   2.420000
6         2      2        4      2   1.000000
7         2      3        1      2   1.000000
8         2      4        2      1   1.500000
9         2      4        3      2  11.500000
10        3      1        2      2   4.366667
11        3      1        3      1   2.000000
12        3      2        4      1   7.633333
13        3      3        1      2   1.000000
14        4      1        3      1   8.033333
15        4      2        4      2   1.333333
16        4      3        1      1   1.000000
17        4      4        2      1   9.633333
18        5      2        4      2   2.000000
19        5      3        1      1   2.000000
20        5      4        2      1

,Recurso,Avión,Destino,Viaje,Unidades
3,1,3,1,2,1.600000
7,2,3,1,2,1.000000
13,3,3,1,2,1.000000
16,4,3,1,1,1.000000
19,5,3,1,1,2.000000
10,3,1,2,2,4.366667
8,2,4,2,1,1.500000
4,1,4,2,1,2.440000
20,5,4,2,1,3.000000
17,4,4,2,1,9.633333


Finalmente se va a identificar como los cambios en los multiplicadores en los destinos afectan la asignación de recursos para estos mismos destinos. En este caso se van a intercambiar multiplicadores de el destino A y el destino C. Esto se hace para evidenciar el mayor impacto sin desnaturalizar el problema (ya que se considera que si se experimenta con valores aleatorios esto puede alterar el análisis). 

Se va a analizar el comportamiento del destino uno. Para la solución original el destino uno tiene asignadas las siguientes unidades en recursos:
Para el destino 1 
- Recurso 1: 1.6
- Recurso 2: 1
- Recurso 3: 1
- Recurso 4: 1
- Recurso 5: 2

Para el destino 3:
- Recurso 1: 3.2
- Recurso 2: 11.5
- Recurso 3: 2
- Recurso 4: 8.03
- Recurso 5: 33

Ahora se evalua para la solución con los multiplicadores alterados.
Para el destino 1:
- Recurso 1: 1.6
- Recurso 2: 1
- Recurso 3: 1
- Recurso 4: 1
- Recurso 5: 2

Para el destino 3:
- Recurso 1: 3.2
- Recurso 2: 11.5
- Recurso 3: 2
- Recurso 4: 8.033
- Recurso 5: 33

Con este cambio se puede observar que los multiplicadores no tienen influencia alguna en la distribución final al menos para los destinos 1 y 3. El único cambio evidenciado fue la distribución de estos vuelos. Ya que para el destino 3 la distribución de los vuelos fueron diferentes.
